# Checking the Safety of a Marlowe Contract

Although the Marlowe language has strong safety guarantees, it is nevertheless possible to create unsafe Marlowe contracts on the Cardano blockchain because some valid Marlowe contract violate the blockchain's ledger rules.

Three Marlowe tools are available for checking the safety of a contract before submitting it to the blockchain:
- Marlowe Runtime's REST endpoint `POST /contracts` returns a safety report in its response.
- Marlowe Runtime's CLI command `marlowe-runtime-cli create` prints the safety report.
- Marlowe CLI's command `marlowe-cli run analyze` prints a similar safety report.

In this tutorial we use the first method, but all three methods would yield the same results.

***It is important to note that Marlowe Runtime does not prevent one from submitting unsafe contracts. It is the responsibility of the client to review the safety errors and decide whether to submit the contract to the blockchain.*** Safety errors are marked as "fatal" to indicate whether the problems are so severe that the contract would not operate on the blockchain.

*Also note that Marlowe's safety-analysis tools have not been audited, so it is advisable to test all of the execution paths of a Marlowe contract on a test network before running the contract on the main network.*

Learn more about Marlowe safety from these resources:
- [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md)
- [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md)

A [video of this safety-check lesson](https://youtu.be/3wb6qOTuBEY) is available.

## Preliminaries

See [Preliminaries](../../docs/preliminaries.md) for information on setting up one's environment for using this tutorial.

The first step is to check we have all the required tools and environment variables available to the notebook. 

In [1]:
export SCRIPTS=../../scripts
export KEYS=../../keys
source $SCRIPTS/check-tools-and-env.sh

########################
## Check CLI commands ##
########################

The following required programs are available in the shell:
  * jq
  * json2yaml
  * marlowe-cli
  * marlowe-runtime-cli
  * cardano-cli
  * cardano-address
  * cardano-wallet

#########################
## Check required envs ##
#########################

The following environment variables are available in the shell:
  * CARDANO_NODE_SOCKET_PATH = node.socket
  * MARLOWE_RT_HOST = 192.168.0.12
  * MARLOWE_RT_PORT = 13700
  * MARLOWE_RT_WEBSERVER_HOST = 192.168.0.12
  * MARLOWE_RT_WEBSERVER_PORT = 13780
  * MARLOWE_RT_WEBSERVER_URL = http://192.168.0.12:13780

###################
## Check Network ##
###################

The NETWORK is set to preprod
CARDANO_TESTNET_MAGIC = 1
CARDANO_SCAN_URL = https://preprod.cardanoscan.io
MARLOWE_SCAN_URL = https://preprod.marlowescan.com


## First example

We create a contract that contains numerous errors in its tokens, names, and addresses.

In [2]:
yaml2json << EOI > contract-1.json
when:
- case:
    deposits: 1
    into_account:
      address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j  # A testnet address.
    of_token:
      currency_symbol: 14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be9711   # Policy ID too short.
      token_name: This token name is longer than the ledger rules allow.        # Token name too long.
    party:
      address: addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h       # A mainnet address.
  then: close
- case:
    choose_between:
    - from: 1
      to: 2
    for_choice:
      choice_name: A Choice
      choice_owner:
        role_token: This role name is longer than the ledger rules allow.       # Role name too long.
  then: close
timeout: $((1000 * (`date -u +%s` + 1 * 60 * 60)))
timeout_continuation: close
EOI

Now create the JSON request for the Marlowe Runtime `POST /contracts` endpoint.

In [3]:
yaml2json << EOI > request-1.json
version: v1
contract: $(cat contract-1.json)
minUTxODeposit: 1200000
roles:
    An Extra Role: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j  # Not present in the contract.
metadata: {}
tags: {}
EOI
cat request-1.json | jq

{
  "contract": {
    "timeout": 1693257312000,
    "timeout_continuation": "close",
    "when": [
      {
        "case": {
          "deposits": 1,
          "into_account": {
            "address": "addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"
          },
          "of_token": {
            "currency_symbol": "14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be9711",
            "token_name": "This token name is longer than the ledger rules allow."
          },
          "party": {
            "address": "addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h"
          }
        },
        "then": "close"
      },
      {
        "case": {
          "choose_between": [
            {
              "from": 1,
              "to": 2
            }
          ],
          "for_choice": {
            "choice_name": "A Choice",
            "choice_owner": {
              "role_token": "This role name is longer than the ledger rules allow."
            }
          }
 

Now call the `POST /contracts` endpoint and receive the response, which contains several reports of safety errors.

In [4]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j" \
  -d @request-1.json \
  -o response-1.json \
  -sS
json2yaml response-1.json

links:
  contract: contracts/860dbbf717702fbb9ff31194235ef20fd51be839fff0d68ca8763045b38d211e%231
resource:
  contractId: 860dbbf717702fbb9ff31194235ef20fd51be839fff0d68ca8763045b38d211e#1
  safetyErrors:
  - detail: This role name is present in the contract, but that role token was not
      specified for minting.
    error: MissingRoleToken
    fatal: true
    role-name: This role name is longer than the ledger rules allow.
  - detail: This role token was specified for minting, but that role is not present
      in the contract.
    error: ExtraRoleToken
    fatal: false
    role-name: An Extra Role
  - detail: This role name is longer than the 32 bytes allowed by the ledger rules.
    error: RoleNameTooLong
    fatal: true
    role-name: This role name is longer than the ledger rules allow.
  - currency-symbol: 14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be9711
    detail: This currency symbol is not the 28-bytes required by the ledger rules.
    error: InvalidCurrencySymbol
   

There are seven safety errors, which we review individually.

In [5]:
jq -r '.resource.safetyErrors | .[] | .error' response-1.json

MissingRoleToken
ExtraRoleToken
RoleNameTooLong
InvalidCurrencySymbol
WrongNetwork
InconsistentNetworks
TransactionValidationError


### `MissingRoleToken`

The role name `This role name is longer than the ledger rules allow.` is present in the contract, but the creation request does not mention that role.

In [6]:
jq '.resource.safetyErrors[0]' response-1.json | json2yaml

detail: This role name is present in the contract, but that role token was not specified
  for minting.
error: MissingRoleToken
fatal: true
role-name: This role name is longer than the ledger rules allow.


### `ExtraRoleToken`

The role name `An Extra Role` is mentioned in the creation request, but not used in the contract itself.

In [7]:
jq '.resource.safetyErrors[1]' response-1.json | json2yaml

detail: This role token was specified for minting, but that role is not present in
  the contract.
error: ExtraRoleToken
fatal: false
role-name: An Extra Role


### `RoleNameTooLong`

The role `This role name is longer than the ledger rules allow.` exceeds the 32-bytes allowed by the ledger rules.

In [8]:
jq '.resource.safetyErrors[2]' response-1.json | json2yaml

detail: This role name is longer than the 32 bytes allowed by the ledger rules.
error: RoleNameTooLong
fatal: true
role-name: This role name is longer than the ledger rules allow.


### `InvalidCurrencySymbol`

The hexadecimal bytes `14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be9711` are not a valid currency symbol, because they are too short.

In [9]:
jq '.resource.safetyErrors[3]' response-1.json | json2yaml

currency-symbol: 14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be9711
detail: This currency symbol is not the 28-bytes required by the ledger rules.
error: InvalidCurrencySymbol
fatal: true


### `WrongNetwork`

The contract mentions a `mainnet` address, but Marlowe Runtime is connected to the `preprod` testnet.

In [10]:
jq '.resource.safetyErrors[4]' response-1.json | json2yaml

detail: The contract contains addresses that are do not match the network on which
  it will be executed.
error: WrongNetwork
fatal: true


### `InconsistentNetworks`

The contract contains a mixture of mainnet and testnet addresses.

In [11]:
jq '.resource.safetyErrors[5]' response-1.json | json2yaml

detail: The contract contains both mainnet and testnet addresses.
error: InconsistentNetworks
fatal: true


### `TransactionValidationError`

The creation transaction would fail validation on the node if it were submitted. The node would not provide a detailed justification for why the transaction fails to validate, so neither can Marlowe Runtime. In this particular case, it is the presence of the illegal token that causes the failure. The safety report lists all of the details of the potential future transaction that would exhibit the validation failure.

In [12]:
jq '.resource.safetyErrors[6]' response-1.json | json2yaml

detail: This transaction fails to validate on the ledger.
error: TransactionValidationError
fatal: true
message: ToCardanoError
transaction:
  contract:
    timeout: 1693257312000
    timeout_continuation: close
    when:
    - case:
        deposits: 1
        into_account:
          address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
        of_token:
          currency_symbol: 14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be9711
          token_name: This token name is longer than the ledger rules allow.
        party:
          address: addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h
      then: close
    - case:
        choose_between:
        - from: 1
          to: 2
        for_choice:
          choice_name: A Choice
          choice_owner:
            role_token: This role name is longer than the ledger rules allow.
      then: close
  input:
    tx_inputs:
    - input_from_party:
        address: addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pv

## Second example

We create a contract that exhibits a few more safety problems.

In [13]:
yaml2json << EOI > contract-2.json
when:
- case:
    deposits: 1
    into_account:
      address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
    of_token:
      currency_symbol: c02699be97113914696a4676909f4e3cb1f2e60e2e08e5abed70caf5
      token_name: This token name is longer than the ledger rules allow.  # Token name too long.
    party:
      role_token: ''
  then: close
- case:
    deposits: 1
    into_account:
      address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
    of_token:
      currency_symbol: ''
      token_name: Impossible to mint.
    party:
      role_token: ''
  then: close
timeout: $((1000 * (`date -u +%s` + 1 * 60 * 60)))
timeout_continuation: close
EOI

Now create the JSON request for the Marlowe Runtime `POST /contracts` endpoint.

In [14]:
yaml2json << EOI > request-2.json
version: v1
contract: $(cat contract-2.json)
minUTxODeposit: 1200000
metadata: {}
tags: {}
EOI
cat request-2.json | jq

{
  "contract": {
    "timeout": 1693257317000,
    "timeout_continuation": "close",
    "when": [
      {
        "case": {
          "deposits": 1,
          "into_account": {
            "address": "addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"
          },
          "of_token": {
            "currency_symbol": "c02699be97113914696a4676909f4e3cb1f2e60e2e08e5abed70caf5",
            "token_name": "This token name is longer than the ledger rules allow."
          },
          "party": {
            "role_token": ""
          }
        },
        "then": "close"
      },
      {
        "case": {
          "deposits": 1,
          "into_account": {
            "address": "addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"
          },
          "of_token": {
            "currency_symbol": "",
            "token_name": "Impossible to mint."
          },
          "party": {
            "role_token": ""
          }
        },
        "then": "close"
     

Requesting that Marlowe Runtime build the creation transaction results in five safety errors. We discuss the first three.

In [15]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j" \
  -d @request-2.json \
  -o response-2.json \
  -sS
jq -r '.resource.safetyErrors | .[] | .error' response-2.json

MissingRolesCurrency
InvalidToken
TokenNameTooLong
TransactionValidationError
TransactionValidationError


### `MissingRolesCurrency`

We failed to specify the roles currency in the request body.

In [16]:
jq '.resource.safetyErrors[0]' response-2.json | json2yaml

detail: Roles are present in the contract, but no roles currency was specified.
error: MissingRolesCurrency
fatal: true


### `InvalidToken`

It is impossible for a non-ada token to exist for the blank currency symbol.

In [17]:
jq '.resource.safetyErrors[1]' response-2.json | json2yaml

detail: This token associates a name with the ada currency symbol.
error: InvalidToken
fatal: true
token:
  currency_symbol: ''
  token_name: Impossible to mint.


### `TokenNameTooLong`

The name `This token name is longer than the 32 bytes allowed by the ledger rules.` is too long for a token.

In [18]:
jq '.resource.safetyErrors[2]' response-2.json | json2yaml

detail: This token name is longer than the 32 bytes allowed by the ledger rules.
error: TokenNameTooLong
fatal: true
token-name: This token name is longer than the ledger rules allow.


## Third example

We use the contract from the previous example, but the creation transaction specifies a roles currency policy ID that is too short.

In [19]:
yaml2json << EOI > request-3.json
version: v1
contract: $(cat contract-2.json)
minUTxODeposit: 1200000
roles: a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be971139  # Not a valid policy ID.
metadata: {}
tags: {}
EOI
cat request-3.json | jq

{
  "contract": {
    "timeout": 1693257317000,
    "timeout_continuation": "close",
    "when": [
      {
        "case": {
          "deposits": 1,
          "into_account": {
            "address": "addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"
          },
          "of_token": {
            "currency_symbol": "c02699be97113914696a4676909f4e3cb1f2e60e2e08e5abed70caf5",
            "token_name": "This token name is longer than the ledger rules allow."
          },
          "party": {
            "role_token": ""
          }
        },
        "then": "close"
      },
      {
        "case": {
          "deposits": 1,
          "into_account": {
            "address": "addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"
          },
          "of_token": {
            "currency_symbol": "",
            "token_name": "Impossible to mint."
          },
          "party": {
            "role_token": ""
          }
        },
        "then": "close"
     

Such an invalid roles currency results in an HTTP error response.

In [20]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j" \
  -d @request-3.json \
  -iSs

HTTP/1.1 400 Bad Request
Transfer-Encoding: chunked
Date: Mon, 28 Aug 2023 20:15:20 GMT
Server: Warp/3.3.24

Error in $.roles: invalid bytestring size

## Fourth example

We create the simplest contract, which has no roles or other activity.

In [21]:
cat << EOI > contract-4.json
"close"
EOI

Now create the JSON request for the Marlowe Runtime `POST /contracts` endpoint.

In [22]:
yaml2json << EOI > request-4.json
version: v1
contract: $(cat contract-4.json)
minUTxODeposit: 1200000
roles: 14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be971139
metadata: {}
tags: {}
EOI
cat request-4.json | jq

{
  "contract": "close",
  "metadata": {},
  "minUTxODeposit": 1200000,
  "roles": "14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be971139",
  "tags": {},
  "version": "v1"
}


Requesting that Marlowe Runtime build the creation transaction results in a safety error.

In [23]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j" \
  -d @request-4.json \
  -o response-4.json \
  -Ss
jq -r '.resource.safetyErrors | .[] | .error' response-4.json

ContractHasNoRoles


### `ContractHasNoRoles`

A roles currency symbol was specified for a contract that does not contain any roles.

In [24]:
jq '.resource.safetyErrors[0]' response-4.json | json2yaml

detail: No roles are present in the contract, but a roles currency was specified.
error: ContractHasNoRoles
fatal: false


## Fifth example

We create a contract that obeys the ledger rules but which generates warnings because of Marlowe semantics rules: in this case, the contract tries to pay more ada than was deposited. This contract will run on the blockchain, but the warnings indicate that it may contain design mistakes or not adhere to best design practices for Marlowe contracts.

In [25]:
yaml2json << EOI > contract-5.json
when:
- case:
    deposits: 10000000
    into_account:
      role_token: My Role
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: My Role
  then:
    from_account:
      role_token: My Role
    pay: 20000000
    then: close
    to:
      party:
        role_token: My Role
    token:
      currency_symbol: ''
      token_name: ''
timeout: $((1000 * (`date -u +%s` + 1 * 60 * 60)))
timeout_continuation: close
EOI

Now create the JSON request for the Marlowe Runtime `POST /contracts` endpoint.

In [26]:
yaml2json << EOI > request-5.json
version: v1
contract: $(cat contract-5.json)
minUTxODeposit: 1200000
roles: 99be97113914696a4676909f4e3cb1f2e60e2e08e5abed70caf5c026
metadata: {}
tags: {}
EOI
cat request-5.json | jq

{
  "contract": {
    "timeout": 1693257323000,
    "timeout_continuation": "close",
    "when": [
      {
        "case": {
          "deposits": 10000000,
          "into_account": {
            "role_token": "My Role"
          },
          "of_token": {
            "currency_symbol": "",
            "token_name": ""
          },
          "party": {
            "role_token": "My Role"
          }
        },
        "then": {
          "from_account": {
            "role_token": "My Role"
          },
          "pay": 20000000,
          "then": "close",
          "to": {
            "party": {
              "role_token": "My Role"
            }
          },
          "token": {
            "currency_symbol": "",
            "token_name": ""
          }
        }
      }
    ]
  },
  "metadata": {},
  "minUTxODeposit": 1200000,
  "roles": "99be97113914696a4676909f4e3cb1f2e60e2e08e5abed70caf5c026",
  "tags": {},
  "version": "v1"
}


Requesting that Marlowe Runtime build the creation transaction results in a safety error.

In [27]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j" \
  -d @request-5.json \
  -o response-5.json \
  -sS
jq -r '.resource.safetyErrors | .[] | .error' response-5.json

TransactionWarning


### `TransactionWarning`

Some application of inputs to the contract can trigger a warning from Marlowe semantics. Such a warning will also be visible on the "Warnings" tab of Marlowe Playground. The safety report lists all of the details of the potential future transaction that would exhibit the warning.

In [28]:
jq '.resource.safetyErrors[0]' response-5.json | json2yaml

detail: A Marlowe semantics warning is reported for this transaction.
error: TransactionWarning
fatal: false
transaction:
  contract:
    timeout: 1693257323000
    timeout_continuation: close
    when:
    - case:
        deposits: 10000000
        into_account:
          role_token: My Role
        of_token:
          currency_symbol: ''
          token_name: ''
        party:
          role_token: My Role
      then:
        from_account:
          role_token: My Role
        pay: 20000000
        then: close
        to:
          party:
            role_token: My Role
        token:
          currency_symbol: ''
          token_name: ''
  input:
    tx_inputs:
    - input_from_party:
        role_token: My Role
      into_account:
        role_token: My Role
      of_token:
        currency_symbol: ''
        token_name: ''
      that_deposits: 10000000
    tx_interval:
      from: 0
      to: 0
  output:
    contract: close
    payments:
    - amount: 10000000
      payment_from:


## Sixth example

We create a contract that would exceed the Plutus execution cost limit on the Cardano blockchain.

In [29]:
yaml2json << EOI > contract-6.json
timeout: $((1000 * (`date -u +%s` + 1 * 60 * 60)))
timeout_continuation: close
when:
- case:
    deposits: 5000000
    into_account:
      role_token: Alice
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
  then:
    timeout: $((1000 * (`date -u +%s` + 2 * 60 * 60)))
    timeout_continuation: close
    when:
    - case:
        deposits: 5000000
        into_account:
          role_token: Bob
        of_token:
          currency_symbol: ''
          token_name: ''
        party:
          address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
      then:
        timeout: $((1000 * (`date -u +%s` + 3 * 60 * 60)))
        timeout_continuation: close
        when:
        - case:
            deposits: 5000000
            into_account:
              role_token: Charlie
            of_token:
              currency_symbol: ''
              token_name: ''
            party:
              address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
          then:
            timeout: $((1000 * (`date -u +%s` + 4 * 60 * 60)))
            timeout_continuation: close
            when:
            - case:
                deposits: 5000000
                into_account:
                  role_token: Dave
                of_token:
                  currency_symbol: ''
                  token_name: ''
                party:
                  address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
              then:
                timeout: $((1000 * (`date -u +%s` + 5 * 60 * 60)))
                timeout_continuation: close
                when:
                - case:
                    deposits: 5000000
                    into_account:
                      role_token: Eve
                    of_token:
                      currency_symbol: ''
                      token_name: ''
                    party:
                      address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
                  then:
                    timeout: $((1000 * (`date -u +%s` + 6 * 60 * 60)))
                    timeout_continuation: close
                    when:
                    - case:
                        notify_if: true
                      then: close
EOI

Now create the JSON request for the Marlowe Runtime `POST /contracts` endpoint.

In [30]:
yaml2json << EOI > request-6.json
version: v1
contract: $(cat contract-6.json)
minUTxODeposit: 1200000
roles: 99be97113914696a4676909f4e3cb1f2e60e2e08e5abed70caf5c026
metadata: {}
tags: {}
EOI
cat request-6.json | jq

{
  "contract": {
    "timeout": 1693257328000,
    "timeout_continuation": "close",
    "when": [
      {
        "case": {
          "deposits": 5000000,
          "into_account": {
            "role_token": "Alice"
          },
          "of_token": {
            "currency_symbol": "",
            "token_name": ""
          },
          "party": {
            "address": "addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"
          }
        },
        "then": {
          "timeout": 1693260928000,
          "timeout_continuation": "close",
          "when": [
            {
              "case": {
                "deposits": 5000000,
                "into_account": {
                  "role_token": "Bob"
                },
                "of_token": {
                  "currency_symbol": "",
                  "token_name": ""
                },
                "party": {
                  "address": "addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"
     

Requesting that Marlowe Runtime build the creation transaction results in safety errors.

In [31]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j" \
  -d @request-6.json \
  -o response-6.json \
  -sS
jq -r '.resource.safetyErrors | .[] | .error' response-6.json

TransactionValidationError
TransactionValidationError


### `TransactionValidationError`

We saw this error before in the first example where the Cardano node would have rejected the transaction during phase-one validation. Here we have a phase-two (Plutus) validation failure because a transaction is too costly to execute. Note that the failing transaction is not the creation transaction, but a transaction in the latter stages of the contract's execution. The safety report lists all of the details of the potential future transaction that would exhibit the validation failure.

In [32]:
jq '.resource.safetyErrors | .[0]' response-6.json

{
  "detail": "This transaction fails to validate on the ledger.",
  "error": "TransactionValidationError",
  "fatal": true,
  "message": "BalancingError \"TxBodyScriptExecutionError [(ScriptWitnessIndexTxIn 1,ScriptErrorEvaluationFailed (CekError An error has occurred:  User error:\\nThe machine terminated part way through evaluation due to overspending the budget.\\nThe budget when the machine terminated was:\\n({ cpu: 6343692948\\n| mem: -1158\\n})\\nNegative numbers indicate the overspent budget; note that this only indicatessthe budget that was needed for the next step, not to run the program to completion.) [\\\"Data decoded successfully\\\",\\\"Redeemer decoded successfully\\\",\\\"Script context decoded successfully\\\"])]\"",
  "transaction": {
    "contract": {
      "timeout": 1693275328000,
      "timeout_continuation": "close",
      "when": [
        {
          "case": {
            "notify_if": true
          },
          "then": "close"
        }
      ]
    },
    "in

The `.message` field of the safety error provides the details from the Plutus interpretter.

In [33]:
jq '.resource.safetyErrors | .[0].message' response-6.json

"BalancingError \"TxBodyScriptExecutionError [(ScriptWitnessIndexTxIn 1,ScriptErrorEvaluationFailed (CekError An error has occurred:  User error:\\nThe machine terminated part way through evaluation due to overspending the budget.\\nThe budget when the machine terminated was:\\n({ cpu: 6343692948\\n| mem: -1158\\n})\\nNegative numbers indicate the overspent budget; note that this only indicatessthe budget that was needed for the next step, not to run the program to completion.) [\\\"Data decoded successfully\\\",\\\"Redeemer decoded successfully\\\",\\\"Script context decoded successfully\\\"])]\""


### `SafetyAnalysisTimeout`

Note that a `SafetyAnalysisTimeout` error might be reported if the Marlowe contract is so large that the Marlowe Runtime server runs out of time analyzing the contract. (Marlowe Runtime imposes a time limit on safety analysis in order to mitigate against denial-of-service attacks.) In this situation, one can use `marlowe-cli run analyze` to analyze the contract because `marlowe-cli` does not impose a time limit for the analysis.

## Other safety errors

The examples above demonstrate all of the commonly occuring Marlowe safety errors that would be detected by Marlowe Runtime.

If the contract is built using `marlowe-cli` instead of Marlowe Runtime, then a few additional safety errors might occur. Several of these relate to the initial Marlowe state, which Marlowe Runtime safely constructs but which is not constrained by Marlowe CLI.
- `NonPositiveBalance`: An initial account balance is not positive.
- `DuplicateAccount`: A duplicate account entry is present in the intial Marlowe state.
- `DuplicateChoice`: A duplicate choice entry is present in the initial Marlowe state.
- `DuplicateBoundValue`: A duplicate bound-value entry is present in the initial Marlowe state.

Other safety errors relate to heuristic analysis performed by Marlowe CLI but not by Marlowe Runtime. (They are not necessary for Marlowe Runtime because Marlowe Runtime always attempts to execute every possible transaction for the contract and so would catch these errors. In Marlowe CLI, attempting to execute every transaction is optional.)
- `MaximumValueMayExceedProtocol`: Too many tokens might be stored at some point in the contract.
- `TransactionSizeMayExceedProtocol`: The transaction size (in bytes) might be too large.
- `TransactionCostMayExceedProtocol` : The transaction's execution cost might be too high.
- `MissingContinuation`: The contract is missing a continuation not present in its continuation map.